# JennAI Presentation Installation Guide

This notebook compiles the comprehensive Flask website structure as described in `presentation_installation.md`. 

The goal is to create a Flask website that mimics the elemental structure of `https://ambiq.ai/`, populated with JennAI's brand theme, visual assets, and mission/vision content.

## Architecture Overview

This installation process implements:
1. **Project Directory Structure** - Proper Flask app organization
2. **Brand Asset Integration** - Copy assets from `src/presentation/brand/`
3. **SCSS Theme Compilation** - Convert brand theme to CSS
4. **Template Setup** - Deploy HTML templates with brand integration
5. **Flask Application** - Configure routes and brand content loading
6. **Local Testing** - Run and verify the website

## Prerequisites

- ✅ JennAI conda environment activated (`jennai-root`)
- ✅ SCSS compiler available (Dart Sass)
- ✅ Brand assets located in `src/presentation/brand/`
- ✅ Flask and dependencies installed

In [ ]:
import os
import sys
import subprocess
import shutil
import webbrowser
from pathlib import Path
import time

# Add project root to path for imports
PROJECT_ROOT = Path.cwd() if 'JennAI' in str(Path.cwd()) else Path(__file__).resolve().parent.parent.parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Import project configuration
from config import config

print("🚀 JennAI Presentation Installation")
print("=" * 50)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Brand Directory: {config.BRAND_DIR}")
print(f"Flask App Directory: {config.PRESENTATION_DIR / 'api_server' / 'flask_app'}")
print(f"Current Working Directory: {Path.cwd()}")
print(f"Python Environment: {sys.executable}")
print("=" * 50)

# 1. Project Directory Structure Creation

Verify and create the required Flask application directory structure. The Flask app exists under `src/presentation/api_server/flask_app/` but we need to ensure all static and template directories are properly set up.

In [ ]:
# Define Flask application paths
flask_app_dir = config.PRESENTATION_DIR / "api_server" / "flask_app"
static_dir = flask_app_dir / "static"
templates_dir = flask_app_dir / "templates"

# Required directory structure
required_dirs = [
    static_dir / "css",
    static_dir / "js", 
    static_dir / "img",
    templates_dir,
    flask_app_dir / "routes"
]

print("📁 Creating/Verifying Flask App Directory Structure")
print("-" * 50)

for directory in required_dirs:
    if directory.exists():
        print(f"✅ {directory.relative_to(PROJECT_ROOT)} - EXISTS")
    else:
        directory.mkdir(parents=True, exist_ok=True)
        print(f"✨ {directory.relative_to(PROJECT_ROOT)} - CREATED")

print(f"\n📂 Flask App Structure:")
print(f"└── {flask_app_dir.relative_to(PROJECT_ROOT)}/")
print("    ├── static/")
print("    │   ├── css/")
print("    │   ├── js/")
print("    │   └── img/")
print("    ├── templates/")
print("    └── routes/")

# Verify existing Flask files
flask_files = {
    "Flask App": flask_app_dir / "app.py",
    "Main Routes": flask_app_dir / "routes" / "main_routes.py",
    "Brand Routes": flask_app_dir / "routes" / "brand_routes.py"
}

print(f"\n🔍 Verifying Existing Flask Files:")
for name, path in flask_files.items():
    status = "✅ EXISTS" if path.exists() else "❌ MISSING"
    print(f"  {name}: {status}")

print("\n✅ Directory structure verification complete!")

# 2. Populating Static Assets

Copy brand assets from `src/presentation/brand/` to the Flask static directory. This includes images, favicon files, and other visual assets needed for the website to properly display the JennAI brand.

In [ ]:
# Define brand asset mapping from presentation_installation.md
brand_assets = {
    "jennai-logo.png": "jennai-logo.png",
    "ai.jpg": "person-interacting-ai.jpg", 
    "circuit-dark.jpg": "circuit-dark-bg.jpg",
    "circuit-light.jpg": "circuit-light-bg.jpg",
    "background.jpg": "abstract-wave-bg.jpg",
    "background-light.jpg": "abstract-wave-bg.jpg",  # fallback
    "heart-blackbackground.jpg": "neon-heart.jpg",
    "heart-blueforeground.jpg": "neon-heart.jpg",  # fallback
    "me.jpeg": "your-portrait.jpg",
    "me1.jpg": "your-portrait.jpg",  # fallback
    "person.jpg": "person-interacting-ai.jpg"  # alternative
}

print("🎨 Copying Brand Assets to Flask Static Directory")
print("-" * 50)

# Copy image assets
copied_count = 0
for source_name, dest_name in brand_assets.items():
    source_path = config.BRAND_DIR / source_name
    dest_path = static_dir / "img" / dest_name
    
    if source_path.exists() and not dest_path.exists():
        shutil.copy2(source_path, dest_path)
        print(f"✅ Copied {source_name} → {dest_name}")
        copied_count += 1
    elif source_path.exists():
        print(f"⚡ Exists {source_name} → {dest_name}")
    else:
        print(f"⚠️  Missing {source_name}")

# Copy favicon files
favicon_source = config.BRAND_DIR / "favicon_io"
if favicon_source.exists():
    favicon_files = list(favicon_source.glob("*"))
    for favicon_file in favicon_files:
        if favicon_file.is_file():
            dest_favicon = static_dir / "img" / favicon_file.name
            if not dest_favicon.exists():
                shutil.copy2(favicon_file, dest_favicon)
                print(f"✅ Copied favicon: {favicon_file.name}")
                copied_count += 1
else:
    print("⚠️  Favicon directory not found")

print(f"\n📊 Asset Summary:")
print(f"  📁 Source: {config.BRAND_DIR.relative_to(PROJECT_ROOT)}")
print(f"  📁 Destination: {(static_dir / 'img').relative_to(PROJECT_ROOT)}")
print(f"  📋 Files copied: {copied_count}")

# List all files in static/img
if (static_dir / "img").exists():
    img_files = list((static_dir / "img").glob("*"))
    print(f"  📋 Total files in destination: {len([f for f in img_files if f.is_file()])}")
    
print("\n✅ Brand asset population complete!")

# 3. Compiling SCSS to CSS

Copy the brand theme from `src/presentation/brand/theme.scss` to Flask static CSS directory as `_variables.scss` and compile the SCSS to CSS using the project's compilation script.

In [ ]:
# Copy brand theme to Flask CSS directory
brand_theme = config.BRAND_DIR / "theme.scss"
flask_variables = static_dir / "css" / "_variables.scss"

print("🎨 Setting Up Brand Theme for SCSS Compilation")
print("-" * 50)

if brand_theme.exists():
    shutil.copy2(brand_theme, flask_variables)
    print(f"✅ Copied brand theme:")
    print(f"   From: {brand_theme.relative_to(PROJECT_ROOT)}")
    print(f"   To:   {flask_variables.relative_to(PROJECT_ROOT)}")
else:
    print(f"❌ Brand theme not found at: {brand_theme.relative_to(PROJECT_ROOT)}")

# Use the existing compile_scss.py script
compile_script = PROJECT_ROOT / "admin" / "compile_scss.py"
scss_source = static_dir / "css" / "main.scss"
css_output = static_dir / "css" / "main.css"

print(f"\n🔧 Compiling SCSS to CSS...")
print(f"   Script: {compile_script.relative_to(PROJECT_ROOT)}")
print(f"   Source: {scss_source.relative_to(PROJECT_ROOT)}")
print(f"   Output: {css_output.relative_to(PROJECT_ROOT)}")

if compile_script.exists():
    try:
        # Change to project root for compilation
        original_cwd = os.getcwd()
        os.chdir(PROJECT_ROOT)
        
        result = subprocess.run([
            sys.executable, str(compile_script), "--target", "flask"
        ], capture_output=True, text=True, check=True, timeout=30)
        
        os.chdir(original_cwd)  # Restore original directory
        
        print("✅ SCSS compilation successful!")
        if result.stdout:
            print(f"   Output: {result.stdout.strip()}")
        
        if css_output.exists():
            css_size = css_output.stat().st_size
            print(f"✅ Generated CSS file: {css_size:,} bytes")
            
            # Preview CSS content
            with open(css_output, 'r', encoding='utf-8') as f:
                css_preview = f.read()[:300]  # First 300 chars
            print(f"📄 CSS Preview (first 300 chars):")
            print("-" * 30)
            print(css_preview + "...")
            print("-" * 30)
        else:
            print("❌ CSS file was not generated")
            
    except subprocess.CalledProcessError as e:
        print(f"❌ SCSS compilation failed: {e}")
        if e.stderr:
            print(f"   Error: {e.stderr}")
    except subprocess.TimeoutExpired:
        print("❌ SCSS compilation timed out")
    finally:
        if os.getcwd() != original_cwd:
            os.chdir(original_cwd)
else:
    print(f"❌ Compile script not found: {compile_script.relative_to(PROJECT_ROOT)}")

print("\n✅ SCSS compilation process complete!")

# 4. Setting Up the Flask Application

Verify that the Flask application files are properly configured with brand content loading. The Flask app should integrate with the brand assets and display the mission/vision content.

In [ ]:
# Verify Flask application setup and brand integration
print("🌐 Verifying Flask Application Setup")
print("-" * 50)

# Test Flask app import and brand content loading
try:
    # Ensure we're in the right directory
    original_cwd = os.getcwd()
    os.chdir(PROJECT_ROOT)
    
    # Add Flask app directory to path
    flask_app_path = str(flask_app_dir)
    if flask_app_path not in sys.path:
        sys.path.insert(0, flask_app_path)
    
    # Test Flask app import
    from app import app
    print(f"✅ Flask app imported successfully")
    print(f"   App name: {app.name}")
    print(f"   Debug mode: {app.debug}")
    print(f"   Static folder: {app.static_folder}")
    print(f"   Template folder: {app.template_folder}")
    
    # Test brand content loading
    from routes.main_routes import load_brand_content
    brand_content = load_brand_content()
    
    print(f"\n🎨 Brand Content Integration Test:")
    mission = brand_content.get('mission_statement', '')
    vision = brand_content.get('vision_statement', '')
    problem = brand_content.get('problem_statement', '')
    
    print(f"   Mission: {'✅ Loaded' if mission else '❌ Missing'} ({len(mission)} chars)")
    if mission:
        print(f"     Preview: {mission[:50]}...")
    
    print(f"   Vision: {'✅ Loaded' if vision else '❌ Missing'} ({'HTML' if '<' in vision else 'Text'} format)")
    if vision:
        # Count actual text content, not HTML tags
        import re
        vision_text = re.sub(r'<[^>]+>', '', vision)
        print(f"     Preview: {vision_text[:50]}...")
    
    print(f"   Problem: {'✅ Loaded' if problem else '❌ Missing'} ({'HTML' if '<' in problem else 'Text'} format)")
    if problem:
        problem_text = re.sub(r'<[^>]+>', '', problem)
        print(f"     Preview: {problem_text[:50]}...")
    
    # Test template rendering
    with app.test_client() as client:
        response = client.get('/')
        print(f"\n🌐 Template Rendering Test:")
        print(f"   Status Code: {response.status_code}")
        print(f"   Content Type: {response.content_type}")
        print(f"   Response Size: {len(response.data):,} bytes")
        
        if response.status_code == 200:
            content = response.data.decode('utf-8')
            # Check for key elements
            checks = {
                'JennAI': 'JennAI' in content,
                'CSS Link': 'main.css' in content,
                'Logo': 'jennai-logo.png' in content,
                'Mission': mission[:20] in content if mission else False,
                'Vision Content': len(re.findall(r'vision', content, re.IGNORECASE)) > 0
            }
            
            print(f"   Content Checks:")
            for check, passed in checks.items():
                status = "✅" if passed else "❌"
                print(f"     {status} {check}")
        else:
            print(f"   ❌ Template rendering failed")
    
except ImportError as e:
    print(f"❌ Flask app import failed: {e}")
    print("   Make sure Flask app files exist and are properly configured")
except Exception as e:
    print(f"❌ Flask app test failed: {e}")
finally:
    if original_cwd and os.getcwd() != original_cwd:
        os.chdir(original_cwd)

print("\n✅ Flask application verification complete!")

# 5. Running the Flask Website Locally

Start the Flask development server to test the website. The server will run in the background and can be accessed at `http://localhost:5000`. This section provides code to start the server and check its status.

In [ ]:
# Start Flask development server
import threading
import requests
import socket

def check_port_available(port):
    """Check if a port is available."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) != 0

def start_flask_server():
    """Start Flask server in a separate thread."""
    try:
        os.chdir(flask_app_dir)
        from app import app
        app.run(host='localhost', port=5000, debug=False, use_reloader=False)
    except Exception as e:
        print(f"Server error: {e}")

print("🚀 Starting Flask Development Server")
print("-" * 50)

# Check if port 5000 is available
if not check_port_available(5000):
    print("⚠️  Port 5000 is already in use")
    print("   The Flask server might already be running")
    print("   Or another application is using this port")
else:
    print("✅ Port 5000 is available")
    
    # Start server in background thread
    server_thread = threading.Thread(target=start_flask_server, daemon=True)
    server_thread.start()
    
    print("🔄 Starting server thread...")
    time.sleep(3)  # Give server time to start

# Test server connectivity
print(f"\n🌐 Testing Server Connectivity")
try:
    response = requests.get('http://localhost:5000', timeout=5)
    print(f"✅ Server is running!")
    print(f"   Status: {response.status_code}")
    print(f"   URL: http://localhost:5000")
    print(f"   Response time: {response.elapsed.total_seconds():.2f}s")
    print(f"   Content length: {len(response.content):,} bytes")
    
    # Quick content check
    if 'JennAI' in response.text:
        print("✅ JennAI branding detected in response")
    if 'main.css' in response.text:
        print("✅ CSS stylesheet linked")
    if 'jennai-logo.png' in response.text:
        print("✅ Logo image referenced")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Server connection failed: {e}")
    print("   Server may not have started yet or there's a configuration issue")

print(f"\n📋 Next Steps:")
print("   1. If server is running, proceed to section 6 for browser testing")
print("   2. If server failed, check Flask app configuration")
print("   3. Server runs in background - no need to restart unless there are errors")
print("   4. To stop server: restart the Python kernel or use admin console")

print("\n✅ Flask server startup process complete!")

# 6. Testing the Website in the Browser

Open the local Flask website in the default browser for visual verification. This section will automatically launch the browser and perform final validation checks.

In [ ]:
# Open website in browser and perform final validation
print("🌐 Opening JennAI Website in Browser")
print("-" * 50)

website_url = "http://localhost:5000"

# Final server check before opening browser
try:
    response = requests.get(website_url, timeout=5)
    if response.status_code == 200:
        print(f"✅ Server responding at {website_url}")
        print(f"   Status: {response.status_code} OK")
        
        # Open in default browser
        webbrowser.open(website_url)
        print(f"🌐 Opened {website_url} in default browser")
        
        # Perform final content validation
        content = response.text
        
        print(f"\n🔍 Final Content Validation:")
        validations = {
            "HTML Structure": content.startswith('<!DOCTYPE html'),
            "Title Tag": '<title>' in content and 'JennAI' in content,
            "CSS Stylesheet": 'main.css' in content,
            "JennAI Logo": 'jennai-logo.png' in content,
            "Brand Mission": any(phrase in content.lower() for phrase in ['bridge', 'unyielding', 'mission']),
            "Vision Content": 'vision' in content.lower(),
            "Favicon": 'favicon' in content,
            "Navigation": 'nav' in content.lower(),
            "Footer": 'footer' in content.lower(),
            "JavaScript": '.js' in content
        }
        
        passed = sum(validations.values())
        total = len(validations)
        
        for check, result in validations.items():
            status = "✅" if result else "⚠️ "
            print(f"   {status} {check}")
        
        print(f"\n📊 Validation Summary:")
        print(f"   ✅ Passed: {passed}/{total} checks")
        print(f"   📄 Page size: {len(content):,} characters")
        print(f"   🎨 Brand integration: {'Complete' if passed >= total * 0.8 else 'Partial'}")
        
        if passed >= total * 0.8:
            print(f"\n🎉 Installation Successful!")
            print(f"   🌐 Website URL: {website_url}")
            print(f"   🎨 Brand theme: Applied")
            print(f"   📱 Responsive design: Enabled")
            print(f"   🚀 Ready for development!")
        else:
            print(f"\n⚠️  Installation completed with issues")
            print(f"   Some validations failed - check content and styling")
            
    else:
        print(f"❌ Server returned status {response.status_code}")
        print("   Check Flask app configuration and try again")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Cannot connect to server: {e}")
    print("   Make sure Flask server is running from section 5")

print(f"\n📋 Manual Testing Checklist:")
print("   □ Check that the website loads in the browser")
print("   □ Verify JennAI branding and logo display")
print("   □ Test responsive design on different screen sizes")
print("   □ Confirm mission and vision content is visible")
print("   □ Check that images load properly")
print("   □ Verify navigation and footer elements")
print("   □ Test button interactions and hover effects")

print(f"\n🏁 JennAI Presentation Installation Complete!")
print("=" * 50)